In [3]:
import gzip
import pickle
import tensorflow
import numpy as np
from tensorflow import keras


def evaluate(path, model):
    X, y = pickle.load(gzip.open(path, 'rb'))
    y[y != 0] -= 2
    X = X / 255.
    acc = np.mean(model(X).numpy().argmax(axis=1) == y)
    return acc

In [5]:
path = 'flatland_train.data'
X, y = pickle.load(gzip.open(path, 'rb')) # (10000, 50, 50)
# data pre-processing
y[y != 0] -= 2 # labels from 0 to 4
X = X / 255. # in range [0, 1]

In [6]:
model = keras.models.Sequential()
#model.add(keras.layers.Flatten(input_shape=(50, 50))) # (10000, 2500)
model.add(keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=[50, 50, 1]))
model.add(keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())
#model.add(keras.layers.Dense(32, activation='selu'))
model.add(keras.layers.Dense(16, activation='selu'))
model.add(keras.layers.Dense(5, activation='softmax')) # suskaiciuoja kazka panasaus i tikimybe, labiau tiketinumas nei tikimybe
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='Adam', 
              metrics=['accuracy'])
# if you are using categorical_crossentropy, then it expects one hot encoded output
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 16)        2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 16)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 8464)              0         
                                                                 
 dense (Dense)               (None, 16)                135440    
                                                                 
 dense_1 (Dense)             (None, 5)                 85        
                                                        

In [7]:
model.fit(X, y, epochs = 10) # accuaracy kyla su kiekvienu paleidimu, nes kiekviena karta modelio neperkuria

Epoch 1/10
313/313 [==============================] - 27s 84ms/step - loss: 1.2135 - accuracy: 0.4808
Epoch 2/10
313/313 [==============================] - 26s 83ms/step - loss: 0.8150 - accuracy: 0.6797
Epoch 3/10
313/313 [==============================] - 26s 84ms/step - loss: 0.5238 - accuracy: 0.8179
Epoch 4/10
313/313 [==============================] - 26s 83ms/step - loss: 0.3842 - accuracy: 0.8805
Epoch 5/10
313/313 [==============================] - 26s 84ms/step - loss: 0.2974 - accuracy: 0.9152
Epoch 6/10
313/313 [==============================] - 26s 84ms/step - loss: 0.2286 - accuracy: 0.9431
Epoch 7/10
313/313 [==============================] - 28s 89ms/step - loss: 0.1880 - accuracy: 0.9553
Epoch 8/10
313/313 [==============================] - 26s 84ms/step - loss: 0.1495 - accuracy: 0.9645
Epoch 9/10
313/313 [==============================] - 26s 84ms/step - loss: 0.1268 - accuracy: 0.9670
Epoch 10/10
313/313 [==============================] - 26s 84ms/step - loss: 0.103

In [8]:
model.save('model.h5')

In [9]:
model = keras.models.load_model('model.h5')
evaluate('flatland_train.data', model)

0.9844